In [3]:
from netCDF4 import Dataset, MFDataset
import numpy as np
from datetime import datetime, timedelta
from cftime import num2date, date2num, date2index
import geopandas as gpd
from shapely.geometry import Polygon

In [6]:
# all relevant climate variables. The key is already the the correct key for  MONICA climate jsons
CLIMATE_VARIABLES = { 
    '3': 'tasmin',
    '4':'tas',
    '5': 'tasmax',
    '6': 'pr',
    '8': 'rsds',
    '9': 'sfcWind',
    '12': 'hurs'
    }


In [7]:
nc = Dataset('climate_netcdf/zalf_tasmin_amber_2021_v1-0.nc', 'r')

In [8]:
nc.variables['tasmin'][:, 205, 201]

masked_array(data=[  0.5       ,   2.8       ,   2.6000001 ,   1.5       ,
                     1.8000001 ,   0.3       ,   0.3       ,   0.4       ,
                    -1.3000001 ,  -1.1       ,   2.3       ,   1.4       ,
                     0.4       ,  -0.1       ,  -1.7       ,  -2.3       ,
                    -0.90000004,   1.1       ,   3.9       ,   6.1       ,
                     5.2000003 ,   0.8       ,  -0.3       ,  -1.6       ,
                    -3.6000001 ,   0.        ,   0.5       ,  -0.2       ,
                    -1.5       ,  -8.7       , -11.        ,  -7.6       ,
                    -2.2       ,   0.1       ,   0.7       ,   0.2       ,
                    -2.6000001 ,  -5.5       ,  -7.3       ,  -8.3       ,
                    -5.6       ,  -9.        , -10.5       , -12.8       ,
                   -10.900001  ,  -2.7       ,   1.7       ,   5.        ,
                     5.2000003 ,   2.7       ,   5.7000003 ,   5.2000003 ,
                     4.4 

In [9]:
time_300 = num2date(1, units=nc.variables['time'].units, calendar=nc.variables['time'].calendar)
time_300

In [11]:
date = num2date(5, nc['time'].units, only_use_cftime_datetimes=True)
date.strftime('%Y-%m-%d %H:%M:%S')

In [11]:
iso_str = "2021-01-15"
german_date = "15.01.2021"

In [12]:
date = datetime.fromisoformat(iso_str)
german_date = datetime.strptime(german_date, "%d.%m.%Y")
date, german_date

(datetime.datetime(2021, 1, 15, 0, 0), datetime.datetime(2021, 1, 15, 0, 0))

In [14]:
num = date2num(date, nc['time'].units)
num

In [15]:
date2index(date, nc['time'])

14

In [112]:
start_date = "02/24/2019"
end_date = "13/11/2021"



def get_climate_data_as_json(start_date_str, end_date_str, lat_idx, lon_idx, climate_data_path="./climate_netcdf/"):
    # opening with MFDataset does not work, because time is not an unlimited dimension in the NetCDF files
    start_date = datetime.strptime(start_date_str, "%m/%d/%Y")
    end_date = datetime.strptime(end_date_str, "%m/%d/%Y")
    climate_json = { 
        '3': [],
        '4': [],
        '5': [],
        '6': [],
        '8': [],
        '9': [],
        '12': [],
        }
    


    for year in range(start_date.year, end_date.year + 1):
        for key, value in CLIMATE_VARIABLES.items():
            file_path = f"{climate_data_path}zalf_{value.lower()}_amber_{year}_v1-0.nc"
            print(file_path)
            nc = Dataset(file_path, 'r')
            start_idx = 0
            end_idx = len(nc['time'])
            if year == start_date.year:
                start_idx = date2index(start_date, nc['time'])
            elif year == end_date.year + 1:
                end_idx = date2index(end_date, nc['time'])

            values = list(nc.variables[value][start_idx:end_idx, lat_idx, lon_idx])
           
            climate_json[key].extend(values)
            nc.close()
            print(year, value, key)

    return climate_json

In [113]:
a = get_climate_data_as_json("02/24/2019", "11/13/2022", 600, 300)

./climate_netcdf/zalf_tasmin_amber_2019_v1-0.nc
2019 tasmin 3
./climate_netcdf/zalf_tas_amber_2019_v1-0.nc
2019 tas 4
./climate_netcdf/zalf_tasmax_amber_2019_v1-0.nc
2019 tasmax 5
./climate_netcdf/zalf_pr_amber_2019_v1-0.nc
2019 pr 6
./climate_netcdf/zalf_rsds_amber_2019_v1-0.nc
2019 rsds 8
./climate_netcdf/zalf_sfcwind_amber_2019_v1-0.nc
2019 sfcWind 9
./climate_netcdf/zalf_hurs_amber_2019_v1-0.nc
2019 hurs 12
./climate_netcdf/zalf_tasmin_amber_2020_v1-0.nc
2020 tasmin 3
./climate_netcdf/zalf_tas_amber_2020_v1-0.nc
2020 tas 4
./climate_netcdf/zalf_tasmax_amber_2020_v1-0.nc
2020 tasmax 5
./climate_netcdf/zalf_pr_amber_2020_v1-0.nc
2020 pr 6
./climate_netcdf/zalf_rsds_amber_2020_v1-0.nc
2020 rsds 8
./climate_netcdf/zalf_sfcwind_amber_2020_v1-0.nc
2020 sfcWind 9
./climate_netcdf/zalf_hurs_amber_2020_v1-0.nc
2020 hurs 12
./climate_netcdf/zalf_tasmin_amber_2021_v1-0.nc
2021 tasmin 3
./climate_netcdf/zalf_tas_amber_2021_v1-0.nc
2021 tas 4
./climate_netcdf/zalf_tasmax_amber_2021_v1-0.nc
2021

In [114]:
a['5']

[11.400001,
 15.900001,
 17.800001,
 18.2,
 18.0,
 10.900001,
 9.1,
 12.1,
 13.3,
 8.3,
 15.2,
 13.0,
 9.8,
 11.0,
 14.6,
 7.0,
 10.1,
 9.400001,
 6.7000003,
 11.6,
 12.3,
 14.8,
 8.900001,
 9.5,
 12.1,
 16.0,
 19.0,
 19.6,
 13.0,
 9.6,
 9.0,
 11.8,
 13.6,
 17.2,
 19.9,
 19.800001,
 17.6,
 20.9,
 16.800001,
 9.1,
 9.400001,
 19.1,
 18.9,
 16.6,
 18.6,
 14.6,
 12.0,
 8.7,
 8.5,
 10.3,
 17.9,
 19.7,
 20.6,
 21.7,
 23.9,
 24.4,
 24.300001,
 24.5,
 22.6,
 26.6,
 26.6,
 11.2,
 15.0,
 13.1,
 12.6,
 17.800001,
 19.9,
 18.7,
 14.2,
 8.6,
 11.3,
 12.3,
 13.900001,
 13.3,
 13.7,
 16.4,
 17.300001,
 15.5,
 16.6,
 15.0,
 14.5,
 13.5,
 20.1,
 23.6,
 24.6,
 18.6,
 15.7,
 17.6,
 20.4,
 22.7,
 22.7,
 22.4,
 22.0,
 16.1,
 16.9,
 21.7,
 24.1,
 27.6,
 29.9,
 31.0,
 30.4,
 31.6,
 20.7,
 28.2,
 21.5,
 25.1,
 22.9,
 22.300001,
 19.1,
 24.7,
 27.4,
 24.2,
 20.5,
 26.7,
 29.4,
 30.5,
 25.1,
 25.2,
 26.9,
 27.800001,
 30.7,
 33.7,
 35.3,
 31.5,
 28.6,
 33.2,
 35.7,
 29.7,
 26.300001,
 25.6,
 26.6,
 28.2,
 30.6

In [9]:
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [14]:
date = datetime.now()

In [15]:
date


datetime.datetime(2025, 2, 10, 14, 52, 20, 916596)

In [17]:
dateBefore = date - relativedelta(months=6)

In [19]:
dateBefore.replace(day=1)


datetime.datetime(2024, 8, 1, 14, 52, 20, 916596)

In [20]:
date = (datetime.now() - relativedelta(months=6)).replace(day=1)

In [26]:
d2 = (datetime.now().date() + relativedelta(months=7)).replace(day=1) - relativedelta(days=1)

In [28]:
d2

datetime.date(2025, 8, 31)